# <내가 읽은 글의 작가의 다른 글을 나에게 추천(다른 사람들이 많이 읽은 순서대로)>

In [160]:
def replace_sentences(text):
    text = text.replace(" #", "\n#")
    sentences = text.splitlines()
    
    return sentences


## 1. user_json 만들기

In [161]:
import os

#change path
directory = '/Users/baekseoin/Desktop/업로드용_seoin'

file_dir = directory + '/res/read'
file_list = os.listdir(file_dir)

table = dict()

for file in file_list:
    #print(len(file))
    path = directory + '/res/read/'+str(file)
    with open(path, "r", encoding='utf-8') as fp:
        #print(fp)
        for line in fp:
            lines = replace_sentences(line)
            #print(lines)
            for i in lines:
                A = i.split()
                #if len(A) == 1:
                    #print(A)
                user = A[0]
                read = A[1:]
                #print(user)
                #print(read)
                #print(read.split()) 

                if user in table:
                    for j in read:
                        if j in table[user]:
                            table[user][j] += 1
                        else:
                            table[user][j] = 1
                else:
                    table[user] = dict()
                    for j in read:
                        if j in table[user]:
                            table[user][j] += 1
                        else:
                            table[user][j] = 1

List = []
List.append(table)

def write_pos_review(output_file_name, List):        # def(3) json 형태로 파일 저장 
    
    with open(output_file_name, "w", encoding="utf-8") as output_file:
        for line in List:
            review_str = ujson.dumps(line, ensure_ascii=False)
            print(review_str, file=output_file)

output_file_name = r"user_json.txt"
write_pos_review(output_file_name, List)




## 2. user_json2 만들기 (2/22~3/1 기간 동안 독자가 읽은 글)

In [162]:
import os

#change path
directory = '/Users/baekseoin/Desktop/업로드용_seoin'

new_file_list = []
for f in file_list:
    name = f.split('_')
    if int(name[0]) >= 2019021400:
        new_file_list.append(f)
    
table = dict()

for file in new_file_list:
    #print(len(file))
    path = directory + '/res/read/'+str(file)
    with open(path, "r", encoding='utf-8') as fp:
        #print(fp)
        for line in fp:
            lines = replace_sentences(line)
            #print(lines)
            for i in lines:
                A = i.split()
                #if len(A) == 1:
                    #print(A)
                user = A[0]
                read = A[1:]
                #print(user)
                #print(read)
                #print(read.split()) 

                if user in table:
                    for j in read:
                        if j in table[user]:
                            table[user][j] += 1
                        else:
                            table[user][j] = 1
                else:
                    table[user] = dict()
                    for j in read:
                        if j in table[user]:
                            table[user][j] += 1
                        else:
                            table[user][j] = 1

List = []
List.append(table)

def write_pos_review(output_file_name, List):        # def(3) json 형태로 파일 저장 
    
    with open(output_file_name, "w", encoding="utf-8") as output_file:
        for line in List:
            review_str = ujson.dumps(line, ensure_ascii=False)
            print(review_str, file=output_file)

output_file_name = r"user_json2.txt"
write_pos_review(output_file_name, List)


In [163]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt

In [164]:
dev_users = []
path1 = directory + '/res/predict/' + 'dev.users.txt'
with open(path1,'r') as dev:
    for line in dev:
        line = line.rstrip()
        dev_users.append(line)

In [165]:
test_users = []
path2 = directory + '/res/predict/' + 'test.users.txt'
with open(path2,'r') as test:
    for line in test:
        line = line.rstrip()
        test_users.append(line)
print(len(test_users))

5000


In [166]:
import json
import ujson
from pprint import pprint

USER = dict()
#test 유저가 전체 기간동안 읽은 글 확인
if __name__ == "__main__":
    file_path = 'user_json.txt'
    with open(file_path,'r') as file_name:
        for users in file_name:
            users = ujson.loads(users)
            for key, values in users.items():
                if key in test_users:                    
                    for k,v in values.items():
                        try:
                            USER[key].append(k)
                        except:
                            USER[key] = []
                            USER[key].append(k)


In [167]:
article = dict()
#작가별로 글이 전체 기간동안 몇번이나 조회되었는지
if __name__ == "__main__":
    file_path = 'user_json.txt'
    with open(file_path,'r') as file_name:
        for users in file_name:
            users = ujson.loads(users)
            for key, values in users.items():
                for k,v in values.items():
                    a = k.split('_')
                    writer = a[0]
                    article_n = a[1]
                    try:
                        article[writer][k] +=v
                    except:
                        try:
                            article[writer][k] = dict()
                            article[writer][k] = v
                        except:
                            article[writer] = dict()
                            article[writer][k] = dict()
                            article[writer][k] = v

In [168]:
#튜플로 변경/정렬하기 위해
article2 = dict()
for k,v in article.items():
    article2[k] = []
    for a,b in v.items():
        article2[k].append((a,b))

In [170]:
#정렬/ 각 작가별로 글의 조회수가 많은 순서대로 정렬
import operator

for k,v in article2.items():
    v =  sorted(v, key=operator.itemgetter(1), reverse=True)
    article2[k] = v
    

In [171]:
#각 유저별로 읽은 글의 작가 수 count
USER_writer = dict()
for i,j in USER.items():
    USER_writer[i] = dict()
    for k in j:
        a = k.split('_')
        try:
            USER_writer[i][a[0]] +=1
        except:
            try:
                USER_writer[i][a[0]] =0
                USER_writer[i][a[0]] +=1
            except:
                USER_writer[i]=dict()
                USER_writer[i][a[0]] =0
                USER_writer[i][a[0]] +=1
        
    

In [172]:
#정렬하기 위해 튜플로 변경
USER_writer2 = dict()
for k,v in USER_writer.items():
    USER_writer2[k] = []
    for a,b in v.items():
        USER_writer2[k].append((a,b))

In [173]:
#정렬/ 많이 읽은 작가 순서대로 정렬

for k,v in USER_writer2.items():
    v =  sorted(v, key=operator.itemgetter(1), reverse=True)
    USER_writer2[k] = v
    

In [174]:
#2/22~3/1까지의 데이터만/위와 똑같은 과정이나, 글의 기간이 짧아짐/최신성 반영

USER_ver2 = dict()

if __name__ == "__main__":
    file_path = 'user_json2.txt'
    with open(file_path,'r') as file_name:
        for users in file_name:
            users = ujson.loads(users)
            for key, values in users.items():
                if key in test_users:                    
                    for k,v in values.items():
                        try:
                            USER_ver2[key].append(k)
                        except:
                            USER_ver2[key] = []
                            USER_ver2[key].append(k)



In [175]:
#각 유저별로 작가수 count

USER_writer_ver2 = dict()
for i,j in USER_ver2.items():
    USER_writer_ver2[i] = dict()
    for k in j:
        a = k.split('_')
        try:
            USER_writer_ver2[i][a[0]] +=1
        except:
            try:
                USER_writer_ver2[i][a[0]] =0
                USER_writer_ver2[i][a[0]] +=1
            except:
                USER_writer_ver2[i]=dict()
                USER_writer_ver2[i][a[0]] =0
                USER_writer_ver2[i][a[0]] +=1

In [176]:
#정렬하기 위해 튜플로 변경
USER_writer2_ver2 = dict()
for k,v in USER_writer_ver2.items():
    USER_writer2_ver2[k] = []
    for a,b in v.items():
        USER_writer2_ver2[k].append((a,b))

In [177]:
#정렬

for k,v in USER_writer2_ver2.items():
    v =  sorted(v, key=operator.itemgetter(1), reverse=True)
    USER_writer2_ver2[k] = v
    

In [178]:
#2/22~3/1까지의 데이터만
#작가가 읽은 글을 확인/작가가 아니라 실제 읽은 글

USER_article = dict()
for i,j in USER_ver2.items():
    USER_article[i] = dict()
    for k in j:
        try:
            USER_article[i][k] +=1
        except:
            try:
                USER_article[i][k] =0
                USER_article[i][k] +=1
            except:
                USER_article[i]=dict()
                USER_article[i][k] =0
                USER_article[i][k] +=1

In [179]:
#각 유저에 대해 읽은 작가와 그 작가의 어떤 글을 읽었는지 확인
user_writer_count = dict()

for k,v in USER_article.items():
    user_writer_count [k] = dict()
    for key,value in v.items():
        a = key.split('_')
        try:
            user_writer_count[k][a[0]].append(key)
        except:
            user_writer_count[k][a[0]] = []
            user_writer_count[k][a[0]].append(key)

In [180]:
#가장 인기있는 글을 확인하기 위해/전체기간동안의 독자의 글 방문 count
path4 = 'user_json.txt'

read2 = {}
with open(path4,'r') as F:
    for x in F:
        x = ujson.loads(x)
        for k,v in x.items():
            for q,w in v.items():
                try:
                    read2[q] += w
                except:
                    read2[q] = 0
                    #print(v)
                    read2[q] += w

In [181]:
#가장 많이 읽힌 글 순서대로 정렬
Total = sorted(read2.items(), reverse=True,key=lambda t : t[1])
#print(Total[0])

Total2 = []

for t in Total:
    Total2.append(t[0])

In [182]:
#following 한 작가가 있는지 확인

following_list = dict()

file = r'users.json.txt'

with open(file,'r') as fp:
    for line in fp:
        line = ujson.loads(line)
        following_list[line['id']] = line['following_list']

In [183]:
#삭제된 글 확인

file = r'metadata.json.txt'

delete_article_check = dict() # 여기에는 현재 존재하는 article이 들어감. 삭제되었으면 이 리스트에 없음

with open(file,'r') as fp:
    for i in fp:
        i = ujson.loads(i)
        delete_article_check[i["id"]] = True

        

In [184]:
def check_len(List):
    a = list(set(List))
    return len(a)

In [185]:
#삭제 리스트 확인
def check_delete(x):
    try:
        A = delete_article_check[x]
        return True
    except:
        return False 

In [186]:
file_name = r'recommend.txt'

with open(file_name,'w') as fp:
    for n_count ,dev in enumerate(dev_users):
        person = []
        total_count = 0
        print(dev,file = fp,end = ' ')
        
        #1. 내가 읽은 글 번호 앞/뒤 글 추천
        try:
            read_list = user_writer_count[dev]
            for key, value in read_list.items():

                if len(value) > 1:
                    for i in value:
                        ww = i.split('_')
                        down = ww[0] + '_' +str(int(ww[1])-1)
                        up = ww[0] + '_' +str(int(ww[1])+1)
                        up2 = ww[0] + '_' +str(int(ww[1])+2)
                        if (down not in value) and ((int(ww[1])-1) != 0) and (down not in person) and check_delete(down):
                            total_count +=1
                            if total_count < 100:                                
                                #print(down,file = fp,end = ' ')
                                person.append(down)
                            else:
                                #print(down,file = fp)
                                person.append(down)
                                break
                        if (up not in value) and (up2 not in value) and (up not in person) and check_delete(up): #중복체크 237, 239 일 경우 238을 한 번만 세도록
                            total_count +=1
                            if total_count < 100:                                
                                #print(up,file = fp,end = ' ')
                                person.append(up)
                            else:
                                #print(up,file = fp)
                                person.append(up)
                                break
                if total_count == 100:
                    break
            
                        
        except:
            pass
        
        if total_count == 100:
            tt = check_len(person)
            if tt != 100:
                print(1,dev)
            for n,peo in enumerate(person):
                if n !=99:
                    print(peo,file = fp, end=' ')
                if n == 99:
                    print(peo,file = fp)
                    break
            continue
                                  
        #2. 내가 읽은 작가 리스트 중 많이 읽은 순서대로 추천/기간:2/22~3/1
        if dev in USER_writer2_ver2:
            w_List = USER_writer2_ver2[dev] # 내가 읽은 작가 리스트_많이 읽은 순서대로
            R = 15
            for w in w_List:
                pop_read =article2[w[0]] #해당 작가가 쓴 글, 조회수가 많은 순서대로
                cc = 0
                for i in pop_read:
                    if (i[0] not in USER[dev]) and (i[0] not in person) and check_delete(i[0]): #작가의 글 중 내가 안읽은 글을 추천
                        total_count +=1
                        if total_count <100:
                            #print(i[0],file = fp, end=' ')
                            person.append(i[0])
                        else:
                            #print(i[0],file = fp) 
                            person.append(i[0])
                        if total_count == 100:
                            break                        
                        cc+=1
                        if cc == R:
                            if R >1:
                                R -=1
                            break
                if total_count == 100:
                            break
            if total_count < 100:
                index = 0
                for i in Total2:
                    if (i not in USER[dev]) and (i not in person) and check_delete(i):
                        if index != (99-total_count):
                            #print(i,file=fp,end=' ')
                            person.append(i)
                            index +=1
                        elif index == (99-total_count):
                            #print(i,file = fp)'
                            person.append(i)
                            break
        #3. 내가 읽은 작가 리스트 중 많이 읽은 순서대로 추천/기간:전체기간
        elif dev in USER_writer2:
            w_List = USER_writer2[dev] # 내가 읽은 작가 리스트_많이 읽은 순서대로
            R = 15
            for w in w_List:
                pop_read =article2[w[0]] #해당 작가가 쓴 글, 조회수가 많은 순서대로
                cc = 0
                for i in pop_read:
                    if (i[0] not in USER[dev]) and check_delete(i[0]): #작가의 글 중 내가 안읽은 글을 추천
                        total_count +=1
                        if total_count <100:
                            #print(i[0],file = fp, end=' ')
                            person.append(i[0])
                        else:
                            #print(i[0],file = fp)    
                            person.append(i[0])
                        if total_count == 100:
                            break                        
                        cc+=1
                        if cc == R:
                            if R >1:
                                R -=1
                            break
                if total_count == 100:
                            break
            if total_count < 100:
                index = 0
                for i in Total2:
                    if (i not in USER[dev]) and (i not in person) and check_delete(i):
                        if index != (99-total_count):
                            #print(i,file=fp,end=' ')
                            person.append(i)
                            index +=1
                        elif index == (99-total_count):
                            #print(i,file = fp)
                            person.append(i)
                            break
                            
        #4. 내가 팔로잉한 작가의 글 추천                 
        elif len(following_list[dev]) >0:
            w_List = following_list[dev] 
            R = 15
            for w in w_List:
                try:
                    pop_read =article2[w] #해당 작가가 쓴 글, 조회수가 많은 순서대로
                except:
                    continue
                cc = 0
                for i in pop_read:
                    if i[0] not in person and check_delete(i[0]):
                        total_count +=1
                        if total_count <100:
                            #print(i[0],file = fp, end=' ')
                            person.append(i[0])
                        else:
                            #print(i[0],file = fp)       
                            person.append(i[0])
                        if total_count == 100:
                            break                        
                        cc+=1
                        if cc == R:
                            if R >1:
                                R -=1
                            break
                if total_count == 100:
                            break
            if total_count < 100:
                index = 0
                for i in Total2:
                    if (i not in person) and (check_delete(i)):
                        if index != (99-total_count):
                            #print(i,file=fp,end=' ')
                            person.append(i)
                            index +=1
                        elif index == (99-total_count):
                            #print(i,file = fp)
                            person.append(i)
                            break  
                            
         #5. 모든 사람이 가장 많이 읽은 글 추천                   
        else:                        
            for index,i in enumerate(Total2):
                if index !=99:
                    #print(i,file = fp, end=' ')
                    person.append(i)
                elif index == 99:
                    #print(i,file = fp)
                    person.append(i)
                    break

        for n,peo in enumerate(person):
            if n !=99:
                print(peo,file = fp, end=' ')
            if n == 99:
                print(peo,file = fp)
                break

         